In [1]:
import numpy as np
import ccdredux as ccd
import imfuncs as imf
from astropy.io import fits as pf
from scipy.ndimage import filters
import shutil
from math import fabs
import glob

In [2]:
""" 
Define a function to create a temporary sky frame from
three input frames
"""

def makesky_3(infiles, medians, indices):
    """ Makes a temporary sky file from 3 input files """
    
    tmpdat = pf.getdata(infiles[indices[0]])
    tmpshape = tmpdat.shape
    tmpsky = np.zeros((3,tmpshape[0],tmpshape[1]))
    for i in range(3):
        tmpsky[i,:,:] = pf.getdata(infiles[indices[i]]) / medians[indices[i]]
    outsky = np.median(tmpsky,axis=0)
    del tmpdat,tmpsky
    return outsky

In [3]:
"""
Define a function that does a quick coadd of a list of input files given
pixel offsets between them.  The quick-and-dirty aspect to this processing
is that the function will just do integer pixel shifts.
"""

def quick_coadd(filelist, offsets, outfile):
    
    """ Start by shifting the offsets to be centered on the mean offset """
    dx = offsets[:,0] - (offsets[:,0].mean())
    dy = offsets[:,1] - (offsets[:,1].mean())
    dxrange = (dx.min(),dx.max())
    dyrange = (dy.min(),dy.min())
    
    """ Make a blank image of the appropriate size """
    dat0 = pf.getdata(filelist[0])
    xsize,ysize = dat0.shape[1],dat0.shape[0]
    del dat0
    outxsize = int(xsize + fabs(dxrange[0]) + fabs(dxrange[1]))
    outysize = int(ysize + fabs(dyrange[0]) + fabs(dyrange[1]))
    outim = np.zeros((outysize,outxsize))
    
    """ Insert the data with the appropriate offsets """
    x0 = fabs(dxrange[0])
    y0 = fabs(dyrange[0])
    x1 = (x0 - dx).astype(int)
    x2 = x1 + int(xsize)
    y1 = (y0 - dy).astype(int)
    y2 = y1 + int(ysize)
    print x1,x2
    print y1,y2
    print x2 - x1
    print y2 - y1
    for i in range(len(filelist)):
        tmp = pf.getdata(filelist[i])
        print i, tmp.shape
        try:
            outim[y1[i]:y2[i],x1[i]:x2[i]] += tmp
        except:
            print 'Failed on image %i (%s) with x1=%d,x2=%d,y1=%d,y2=%d' % (i,filelist[i],x1[i],x2[i],y1[i],y2[i])
        del tmp
    pf.PrimaryHDU(outim).writeto(outfile,clobber=True)

In [4]:
""" 
A function that makes copies of the raw data files that we can freely modify.
Unfortunately, something is screwed up with the header in the raw files,
so we are just going to copy over the data, which is non-optimal
"""
def get_raw(sciframes):
    """ 
    Inputs:
      sciframes - list or array containing the frame numbers associated with
                  the observations of the desired object
      lensroot  - rootname for output files
    """
    rawdir = '../Raw_scam/'
    rawroot = 'jun18i00'   
    infiles = []
    for i in sciframes:
        rawname = '%s%s%02d.fits' % (rawdir,rawroot,i)
        workname = 'work_%02d.fits' % i
        infiles.append(workname)
        #shutil.copyfile(rawname,workname)
        data = pf.getdata(rawname,ignore_missing_end=True)
        pf.PrimaryHDU(data).writeto(workname)
        del data
    return infiles

In [17]:
""" Get the raw data (modify for each lens-filter combination) """
sciframes = np.arange(46,59)
lensroot = 'J1618_Kp'
infiles = get_raw(sciframes)
print ''
print infiles

  1�  5�  7�  9a  7  5�  3�  /�  3r  1�  5n  1�  3�  1�  ,�  (�  5�  6�  ,  4� [astropy.io.fits.card]
  3  5  3�  6  4�  7\  7�  5�  8�  5�  3  6  3  4�  20  4�  7u  7�  3�  0� [astropy.io.fits.card]
  2I  *�  4�  8  5�  9�  4G  7j  6,  7U  4�  2v  6�  2�  2  6  5�  4  5Q  3� [astropy.io.fits.card]
  2�  4f  2�  5"  5   2�  6\  5�  5�  4�  5�  8�  4�  3�  0�  2!  5�  2  4L  2� [astropy.io.fits.card]
  /�  2�  4m  7�  9�  4�  6�  3�  3�  9O  4`  8�  7J  1�  91  1N  5t  4�  7K  7 [astropy.io.fits.card]
  5  7�  5U  6=  5  5{  6C  3�  2�  7|  9  /�  1�  ;5  6�  2  5�  7G  9V  8h [astropy.io.fits.card]
  5�  1)  5>  6   3k  4�  9�  <  6]  0�  6�  3a  6�  4�  7&  9d  6z  4�  <	  5 [astropy.io.fits.card]
  5\  5�  3�  6z  7�  6�  5�  7�  4`  7�  9Q  3�  6�  3�  6�  6�  8�  5  ::  8� [astropy.io.fits.card]
  2  :   8v  8�  5L  8>  5/  5�  1w  8x  2z  54  9�  8�  6�  9}  8)  6m  2�  54 [astropy.io.fits.card]
  4-  6�  6'  3  9�  9j  9�  :�  6?  8h  6,  6I  9�  3F  1"  6I


['work_46.fits', 'work_47.fits', 'work_48.fits', 'work_49.fits', 'work_50.fits', 'work_51.fits', 'work_52.fits', 'work_53.fits', 'work_54.fits', 'work_55.fits', 'work_56.fits', 'work_57.fits', 'work_58.fits']


  5O  1�  4o  2  -"  )  5�  7.  ,?  5= [astropy.io.fits.card]
  3C  5'  4*  6t  4�  7Q  7�  6  9�  59  3  5�  3e  5K  2&  5  7�  7�  45  1n [astropy.io.fits.card]
  2~  +  4�  8�  6_  :�  4B  7�  6�  7�  5,  2�  7  2�  2  5�  6d  3�  5�  4 [astropy.io.fits.card]
  3  4f  30  5m  5q  3H  6�  5�  6e  4�  5�  9t  4�  4  0�  2w  5�  2z  4�  3 [astropy.io.fits.card]
  /�  3B  4�  8_  9*  4�  6�  3�  3�  9�  4�  8�  7�  2D  90  1f  5s  5`  7�  6� [astropy.io.fits.card]
  5�  84  5V  6�  5   5�  6B  2�  2{  7�  9  /�  1�  ;7  6�  2!  6  7�  9�  8z [astropy.io.fits.card]
  4  5^  :  <�  6r  1T  6�  3�  6�  5T  6�  9v  6�  4�  <2  5Q [astropy.io.fits.card]
  5#  5=  3  7   7q  6�  5�  8�  4j  7�  9  4  6�  3�  6�  7@  8�  5n  :�  93 [astropy.io.fits.card]
  1�  :s  8�  9  5�  8�  5[  6  2Z  92  2�  5�  9�  8N  6�  :C  8�  7  2�  53 [astropy.io.fits.card]
  3�  6�  6  3�  9�  9K  9�  :�  6L  8�  6�  6�  :E  31  1e  6�  4�  7�  4*  7� [astropy.io.fits.card]
  �a  5�  8�  4F

In [18]:
""" Make the first sky frame """
skyname = '%s_sky1.fits' % lensroot
ccd.median_combine(infiles,skyname,normalize=True)

median_combine: Inputs:
-----------------------
  bias frame: [No bias file]

median_combine: Loading files
-----------------------------
 work_46.fits
 work_47.fits
 work_48.fits
 work_49.fits
 work_50.fits
 work_51.fits
 work_52.fits
 work_53.fits
 work_54.fits
 work_55.fits
 work_56.fits
 work_57.fits
 work_58.fits

median_combine: Getting info on first file
------------------------------------------
Filename: work_46.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       6   (256, 256)   int32   

median_combine: setting up stack for images (HDU 0)
----------------------------------------------------
Stack will have dimensions (13,256,256)
 work_46.fits
    Normalizing work_46.fits by 14138.000000
 work_47.fits
    Normalizing work_47.fits by 14577.000000
 work_48.fits
    Normalizing work_48.fits by 13743.000000
 work_49.fits
    Normalizing work_49.fits by 13784.000000
 work_50.fits
    Normalizing work_50.fits by 12798.000000
 work_51.fi

In [19]:
""" 
Use the sky frame to make an initial bad pixel mask and then
update the sky frame itself to mask out the bad pixels.
"""
sky = imf.Image(skyname)

""" 
Do a 3-sigma clipping on the data and use the resulting clipped
mean and clipped rms to set the criterion for determining bad 
pixels
"""
sky.sigma_clip()
diff = np.fabs((sky.hdu[0].data - sky.mean_clip) / sky.rms_clip)

""" Create the bad pixel mask and write it out """
bpmask = diff>5.
tmp = bpmask.astype(int)
maskname = '%s_mask_sky1.fits' % lensroot
pf.PrimaryHDU(tmp).writeto(maskname,clobber=True)
del tmp

""" Replace the bad pixels with the median value in the image """
skydat = sky.hdu[0].data.copy()
skymed = np.median(skydat)
skydat[bpmask] = skymed

""" Save the result """
sky1v2name = '%s_sky1_v2.fits' % lensroot
pf.PrimaryHDU(skydat).writeto(sky1v2name,clobber=True)


Loading file J1618_Kp_sky1.fits
-----------------------------------------------
Filename: J1618_Kp_sky1.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       6   (256, 256)   float64   
get_wcs: No valid WCS information in file header


In [20]:
""" 
Replace the bad pixels in the input files using a 2-step process:
  1. Replace each of the bad pixels with the overall median value
  2. WAIT FOR NOW ON THIS

"""
for i in infiles:
    hdu = pf.open(i,mode='update')
    data = hdu[0].data
    datmed = np.median(data)
    data[bpmask] = datmed
    hdu.flush()
    del(hdu)


In [21]:
""" Do a running sky subtraction """

""" Start by reading in all the files and calculating their median values """
alldat = np.zeros((len(infiles),bpmask.shape[0],bpmask.shape[1]))
allmed = np.zeros(len(infiles))
index_list = []
for i in range(len(infiles)):
    tmp = pf.getdata(infiles[i])
    allmed[i] = np.median(tmp)
    if i==0:
        index_list.append([0,1,2])
    elif i==(len(infiles)-1):
        index_list.append([i-2,i-1,i])
    else:
        index_list.append([i-1,i,i+1])
for i in range(len(infiles)):
    tmp = pf.getdata(infiles[i])
    tmpsub = tmp - allmed[i] * makesky_3(infiles,allmed,index_list[i])
    outname = 'tmpsub_%02d.fits' % sciframes[i]
    pf.PrimaryHDU(tmpsub).writeto(outname,clobber=True)
        

In [22]:
"""
Create an updated sky frame by:
  1. Replacing the bad pixels in the sky frame with the median level of the image
"""

""" Replace the bad pixels with the median value in the image """
skydat = sky.hdu[0].data.copy()
skymed = np.median(skydat)
skydat[bpmask] = skymed

""" Smooth by quadrant MAY BE A MISTAKE"""
skysmo = np.ones(skydat.shape)
x = [0, 0, 128, 128]
y = [0, 128, 0, 128]
quad = np.ones((4,128,128))
for i in range(4):
    x1 = x[i]
    x2 = x1 + 128
    y1 = y[i]
    y2 = y1 + 128
    tmp = skydat[y1:y2,x1:x2]
    tmpsmo = filters.uniform_filter(tmp,3,cval=1.)
    skysmo[y1:y2,x1:x2] = tmpsmo.copy()

""" Save the result """
pf.PrimaryHDU(skydat).writeto('sky1_v2.fits')

IOError: File 'sky1_v2.fits' already exists.

In [23]:
""" Do the initial sky subtraction using the updated sky frame """
sky1 = pf.getdata(sky1v2name)
skymed = np.median(sky1)
for i in infiles:
    scidat = pf.getdata(i).astype(float)
    scimed = np.median(scidat)
    scidat[bpmask] = scimed
    scimed2 = np.median(scidat)
    diff = scidat - (scimed2 / skymed) * sky1
    outfile = i.replace('work','sub1')
    pf.PrimaryHDU(diff).writeto(outfile)
    print 'Wrote sky-subtracted image (pass 1) to %s' % outfile
    del scidat,diff

Wrote sky-subtracted image (pass 1) to sub1_46.fits
Wrote sky-subtracted image (pass 1) to sub1_47.fits
Wrote sky-subtracted image (pass 1) to sub1_48.fits
Wrote sky-subtracted image (pass 1) to sub1_49.fits
Wrote sky-subtracted image (pass 1) to sub1_50.fits
Wrote sky-subtracted image (pass 1) to sub1_51.fits
Wrote sky-subtracted image (pass 1) to sub1_52.fits
Wrote sky-subtracted image (pass 1) to sub1_53.fits
Wrote sky-subtracted image (pass 1) to sub1_54.fits
Wrote sky-subtracted image (pass 1) to sub1_55.fits
Wrote sky-subtracted image (pass 1) to sub1_56.fits
Wrote sky-subtracted image (pass 1) to sub1_57.fits
Wrote sky-subtracted image (pass 1) to sub1_58.fits


In [16]:
""" Do a quick-and-dirty coadd """
offsets = np.loadtxt('%s_offsets.txt' % lensroot)
subfiles = []
for i in infiles:
    subfiles.append(i.replace('work','sub1'))
quick_coadd(subfiles,offsets,'%s_coadd_quick.fits' % lensroot)

[22 45  1 45 22  0 45  0 22] [278 301 257 301 278 256 301 256 278]
[21  0 22 43  0 43 21  0 43] [277 256 278 299 256 299 277 256 299]
[256 256 256 256 256 256 256 256 256]
[256 256 256 256 256 256 256 256 256]
0 (256, 256)
1 (256, 256)
2 (256, 256)
3 (256, 256)
4 (256, 256)
5 (256, 256)
6 (256, 256)
7 (256, 256)
8 (256, 256)
